In [11]:
from collections import deque
import time

In [39]:
class Customer: # generate unique Guests
    def __init__(self, id, arrival_time, fast_pass):
        self.id = id # unique
        self.fast_pass = fast_pass # 0 for no fast_pass, 1 for have fast_pass
        self.arrival_time = arrival_time # when they enter the queue
        self.start_time = None # when they start the ride
        self.end_time = None # when they finish the ride
        self.waiting_time = None # waiting time before they get served

    def __repr__(self): # representation method 
        return f"Customer {self.id, self. arrival_time, self.fast_pass}"


In [19]:
class RideQueues:
    def __init__(self, ride_time, ride_capacity, current_time):
        self.regular_queue = deque()  
        self.fast_pass_queue = deque()
        self.ride_time = ride_time  # Time of each ride
        self.ride_capacity = ride_capacity # Max number of ppl a ride can take
        self.current_time = current_time  # Simulation time in seconds
        self.total_served = 0  # cumulative number of served customers 

    def advance_time(self):
        # Update the current time based on the ride time
        self.current_time += self.ride_time
        print(f"Time advanced to: {self.current_time} seconds.")

    def add_customer(self, customer):
        if isinstance(customer, Customer):
            if customer.arrival_time <= self.current_time:
                if customer.fast_pass == 1:
                    self.fast_pass_queue.append(customer)
                    print(f"{customer} added to the fast-pass queue at time {self.current_time}.") # for debugtest
                else:
                    self.regular_queue.append(customer)
                    print(f"{customer} added to the regular queue at time {self.current_time}.") # for debugtest
            else:
                print(f"{customer} has not yet arrived at time {self.current_time}.")
        else:
            print("Only Customer objects can be added to the queue.") # for debugtest
        

    def process_queue(self):
        # assumption: ride_capacity is EVEN number; fast-pass vs regular queue each take up 1/2 of ride_capacity
        # operation: take min(1/2*ride_capacity customeres out of fast_pass_queue, len(fast_pass_queue)), the rest from regular_queue
        
        riding_customers = [] # list of customers being served in the current ride
        early_termination = False # Flag for early termination
        
        # take min(1/2*ride_capacity customeres out of fast_pass_queue, len(fast_pass_queue))
        if self.fast_pass_queue: # while fast_pass_queue has customers
            for i in range(min(self.ride_capacity//2, len(self.fast_pass_queue))):
                person = self.fast_pass_queue.popleft()
                riding_customers.append(person) # Get customers in fast_pass_queue
                print(f"{person} from Fast Pass Queue is being served.") # for debugging
                self.total_served += 1
                print("total serving so far:", self.total_served) # for debugging
        
        fast_pass_filled_ride = len(riding_customers) # number of fast_pass customers going to the current ride
        
        # the rest from regular_queue
        if self.regular_queue:
            for i in range(min(self.ride_capacity - fast_pass_filled_ride, len(self.regular_queue))):
                person = self.regular_queue.popleft()
                riding_customers.append(person) # Get customers in regular_queue
                print(f"{person} from Regular Queue is being served.") # for debugging
                self.total_served += 1
                print("total serving so far:", self.total_served) # for debugging
        
        else:
            print("No one is waiting in the queues.")
            early_termination = True
        
        # now we update the time
        for customer in riding_customers: 
            customer.start_time = self.current_time  # When customer starts the ride
            customer.end_time = self.current_time + self.ride_time
            print(f"{customer} started at {customer.start_time} and finished at {customer.end_time}.") # for debugtest
            self.calculate_waiting_time(customer)

        # Create outputs: curr_wait_time, crowd_level
        crowd_level = {'fast_pass_queue':len(self.fast_pass_queue), 
                       'regular_queue':len(self.regular_queue)}
        curr_wait_time = {'fast_pass_queue':(crowd_level['fast_pass_queue']//self.ride_capacity)*self.ride_time, 
                          'regular_queue':(crowd_level['regular_queue']//self.ride_capacity)*self.ride_time}

        # and lastly, we update the current time
        self.current_time += self.ride_time
        print(f"The time now is {self.current_time}.")

        return {
            'riding_customers': riding_customers, # list of customers being served at the current ride
            'early_termination': early_termination, # flag for early termination
            'total_served': self.total_served, # total number of cust served up until that moment
            'current_time': self.current_time,
            'crowd_level': crowd_level, # dictionary
            'curr_wait_time': curr_wait_time # dictionary
        }

    def calculate_waiting_time(self, customer):
        customer.waiting_time = customer.start_time - customer.arrival_time
        print(f"{customer} waited for {customer.waiting_time} seconds.")

    

In [20]:
# TESTING HERE

def test_ride_queues():
    # Create a RideQueues instance
    ride_time = 10  # Each ride takes 10 seconds
    ride_capacity = 6  # Max capacity of the ride is 6 people
    current_time = 0
    ride_queues = RideQueues(ride_time, ride_capacity, current_time)
    

    # Create some customers
    customers = [
        Customer(id=1, arrival_time=0, fast_pass=0),  # Regular customer
        Customer(id=2, arrival_time=1, fast_pass=1),  # Fast-pass customer
        Customer(id=3, arrival_time=2, fast_pass=0),  # Regular customer
        Customer(id=4, arrival_time=3, fast_pass=1),  # Fast-pass customer
        Customer(id=5, arrival_time=4, fast_pass=0),  # Regular customer
        Customer(id=6, arrival_time=5, fast_pass=1),  # Fast-pass customer
        Customer(id=7, arrival_time=6, fast_pass=0),  # Regular customer
        Customer(id=8, arrival_time=7, fast_pass=1),  # Fast-pass customer
    ]

    # Add customers to the queues
    for customer in customers:
        ride_queues.add_customer(customer)

    # Process the queue for the first ride
    curr_list_of_cust = ride_queues.process_queue()
    print("\n--- End of Ride 1 ---\n")

    # update list of remaining customers:
    if curr_list_of_cust:
        customers = [customer for customer in customers if customer not in curr_list_of_cust]


    # Add customers to the queues
    for customer in customers:
        ride_queues.add_customer(customer)

    # Process the queue for the second ride
    curr_list_of_cust = ride_queues.process_queue()
    print("\n--- End of Ride 2 ---\n")

    # update list of remaining customers:
    if curr_list_of_cust:
        customers = [customer for customer in customers if customer not in curr_list_of_cust]


    # Add customers to the queues
    for customer in customers:
        ride_queues.add_customer(customer)

    # Process the queue for the third ride
    curr_list_of_cust = ride_queues.process_queue()
    print("\n--- End of Ride 3 ---\n")

    # update list of remaining customers:
    if curr_list_of_cust:
        customers = [customer for customer in customers if customer not in curr_list_of_cust]


    # Add customers to the queues
    for customer in customers:
        ride_queues.add_customer(customer)

    # Try processing when there are no customers left
    ride_queues.process_queue()
    print("\n--- End of Ride 4 ---\n")


# Run the test cases
test_ride_queues()


Customer (1, 0) added to the regular queue at time 0.
Customer (2, 1) has not yet arrived at time 0.
Customer (3, 0) has not yet arrived at time 0.
Customer (4, 1) has not yet arrived at time 0.
Customer (5, 0) has not yet arrived at time 0.
Customer (6, 1) has not yet arrived at time 0.
Customer (7, 0) has not yet arrived at time 0.
Customer (8, 1) has not yet arrived at time 0.
Customer (1, 0) from Regular Queue is being served.
total serving so far: 1
Customer (1, 0) started at 0 and finished at 10.
Customer (1, 0) waited for 0 seconds.
The time now is 10.

--- End of Ride 1 ---

Customer (1, 0) added to the regular queue at time 10.
Customer (2, 1) added to the fast-pass queue at time 10.
Customer (3, 0) added to the regular queue at time 10.
Customer (4, 1) added to the fast-pass queue at time 10.
Customer (5, 0) added to the regular queue at time 10.
Customer (6, 1) added to the fast-pass queue at time 10.
Customer (7, 0) added to the regular queue at time 10.
Customer (8, 1) add

In [29]:
# Create a RideQueues instance
ride_time = 10  # Each ride takes 10 seconds
ride_capacity = 6  # Max capacity of the ride is 6 people
current_time = 0 # Customisable
ride_queues = RideQueues(ride_time, ride_capacity, current_time)

# Import cust_df ###IMPT, need to link from attendance.ipynb

# For now we create the cust_df manually
import pandas as pd

data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8],
    'arrival_time': [0, 1, 2, 3, 4, 5, 6, 7],
    'fast_pass': [0, 1, 0, 1, 0, 1, 0, 1]
}

cust_df = pd.DataFrame(data)

print(cust_df)


   id  arrival_time  fast_pass
0   1             0          0
1   2             1          1
2   3             2          0
3   4             3          1
4   5             4          0
5   6             5          1
6   7             6          0
7   8             7          1


In [41]:
# Creating a list of Customer instances from df, need to link from attendance.ipynb

# cust_df columns: id, arrival_time, fast_pass

customers = []
for index, row in cust_df.iterrows():
    new_cust = Customer(id=row['id'], arrival_time=row['arrival_time'], fast_pass=row['fast_pass'])
    customers.append(new_cust)

In [48]:
# Trying to define a function that outputs the crowd_level (number of customers in the queue) and waiting_time (expected wait time if one person is to join the queue)

def stat_ride_queues(ride_queues, customers): # parameters are: a RideQueues object and a list
    # Add customers to the queues
    
    early_termination = False 
    process_queue_stat = {}
    while early_termination == False:
        for customer in customers: # add customers to the queue
            ride_queues.add_customer(customer)

        result = ride_queues.process_queue() # process the queue
        curr_list_of_cust = result['riding_customers']
        early_termination = result['early_termination']
        process_queue_stat[result['current_time']] = result

        # For testing
        if result['early_termination']:
            print("Simulation ended early due to no customers in the queue.")
        else:
            print(f"Riding customers this round: {result['riding_customers']}")
            print(f"Total customers served: {result['total_served']}")

        if curr_list_of_cust: # remove served customers out of the list of customers
            customers = [customer for customer in customers if customer not in curr_list_of_cust] 
    
    return process_queue_stat

In [49]:
# Run the test cases
one_ride_test_result = stat_ride_queues(ride_queues, customers)
print(one_ride_test_result)


Customer (1, 0, 0) added to the regular queue at time 120.
Customer (2, 1, 1) added to the fast-pass queue at time 120.
Customer (3, 2, 0) added to the regular queue at time 120.
Customer (4, 3, 1) added to the fast-pass queue at time 120.
Customer (5, 4, 0) added to the regular queue at time 120.
Customer (6, 5, 1) added to the fast-pass queue at time 120.
Customer (7, 6, 0) added to the regular queue at time 120.
Customer (8, 7, 1) added to the fast-pass queue at time 120.
Customer (2, 1, 1) from Fast Pass Queue is being served.
total serving so far: 40
Customer (4, 3, 1) from Fast Pass Queue is being served.
total serving so far: 41
Customer (6, 5, 1) from Fast Pass Queue is being served.
total serving so far: 42
Customer (1, 0, 0) from Regular Queue is being served.
total serving so far: 43
Customer (3, 2, 0) from Regular Queue is being served.
total serving so far: 44
Customer (5, 4, 0) from Regular Queue is being served.
total serving so far: 45
Customer (2, 1, 1) started at 120 